<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/dev/10_diffusion_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook introduce the time dependent equations of electrodynamics.

We will use Maxwell's equations to explore how the charge, potentials, and fields relax to equilibrium.  Refer to Sections 4.1, 4.4, and 4.5 of our textbook for background on the equations for the fields.

Refer to the [`colab_movies.ipynb`](https://github.com/dr-kinder/playground/blob/dev/colab_movies.ipynb) notebook for more explanation of some details of the code.

# Install and Import

In [1]:
from IPython import display

In [2]:
# Need to upgrade matplotlib for some PyVista plotting commands to work.
# Run this cell, then select "Runtime > Restart runtime" from the CoLab menu.
!pip install --upgrade matplotlib

display.clear_output()

In [3]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

display.clear_output()

In [4]:
# This simulation uses real values.
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

display.clear_output()

In [5]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@906a91b"
    import multiphenicsx

display.clear_output()

In [6]:
# Everything should be installed now.
# Import the rest of what we need.

import gmsh

from dolfinx import fem, mesh, plot, la
from dolfinx.io import gmshio
import ufl

from mpi4py import MPI
import petsc4py.PETSc as petsc

import multiphenicsx.io as mpio

import numpy as np
import pyvista

# Initial Conditions

The code in this section is a copy of [06-finite-element-method.ipynb](https://github.com/dr-kinder/electrodynamics/blob/master/week-03/06-finite-element-method.ipynb).  We will use the solution to the electrostatics problem as the initial condition for an electrodynamics problem.

## Model

This is the same model as the original calculation.

In [7]:
# Create a simple rectangular mesh.
length = 10
height = 10
Nx, Ny = 51, 51
extent = [[-length/2, -height/2], [length/2, height/2]]
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, extent, [Nx, Ny], mesh.CellType.triangle)

In [8]:
# Plot the subdomains that FEniCSx has identified.
# There should only be one for this model.
mpio.plot_mesh(domain)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Charge Density

This time, we will define the charge density as a time-dependent function on the grid.

In [9]:
# Define a set of functions on our mesh.
V = dolfinx.fem.FunctionSpace(domain, ("Lagrange", 2))

In [10]:
# Define the charge.
xC = 2
yC = -1
Q = 1
sQ = 0.2
beta = 1/sQ
tau = 1

# This function returns a function that can be interpolated.
def rho_function(t):
    return lambda x: Q * beta / 2 / np.pi * np.exp(-t/tau) * np.exp(-0.5 * beta**2 * ((x[0]-xC)**2 + (x[1] - yC)**2))

# Turn this function definition into a time-dependent function on the mesh.
rho = fem.Function(V)

# Initialize the function for t=0.
rho.interpolate(rho_function(0))

In [11]:
# Now, plot it.
multiphenicsx.io.plot_scalar_field(rho, name="Charge Density", warp_factor=10)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Boundary Conditions

We will set the potential to zero on the boundary to compute the initial potential and fields.  After this, the boundaries will be open.

In [12]:
# Define a function to locate boundaries.
# It will return True for points on the boundary, and False otherwise.
def find_facets(x):
    yes = np.isclose(x[0], -length/2)
    yes += np.isclose(x[0], length/2)
    yes += np.isclose(x[1], -height/2)
    yes += np.isclose(x[1], height/2)
    return yes

# Next, we use this function to tag each cell along the boundary.
tdim = domain.topology.dim
bc_facets = mesh.locate_entities_boundary(
    domain, tdim - 1, find_facets)

# Identify those for which the function was True as
# boundary_dofs = "boundary degrees of freedom"
boundary_dofs = fem.locate_dofs_topological(V, tdim - 1, bc_facets)

# # Now introduce the boundary condition: constant potential on the boundary.
# Define the potential on the boundary.
def uD_function(t):
    return lambda x: 0 * x[0]

# Turn this function definition into a time-dependent function on the mesh.
uD = fem.Function(V)

# Initialize the function for t=0.
uD.interpolate(uD_function(0.0))

# Add these to the list of boundary conditions to be imposed.
# This will apply the uD function to the cells along the boundary.
# It will be updated during each time step.
bcs = [fem.dirichletbc(uD, boundary_dofs)]

## Solve for the Initial Potential

In [13]:
# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = fem.Function(V)
phi_n = fem.Function(V)

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's equation.
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This is how we introduce the charge density.
# It is the right-hand side of Poisson's equation.
L = 4 * ufl.pi * rho * v * ufl.dx

# Put it all together for FEniCSx.
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, u=phi, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc.InsertMode.INSERT, mode=petsc.ScatterMode.FORWARD)

In [14]:
# Plot the solution.
mpio.plot_scalar_field(phi, "Potential", 10)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [15]:
# Define a set of elements for a vector field.
W = dolfinx.fem.VectorFunctionSpace(domain, ("Lagrange", 2))
E = dolfinx.fem.Function(W)

# Compute the gradient as a symbolic expression, then interpolate it onto the mesh.
expr = dolfinx.fem.Expression(ufl.as_vector((-phi.dx(0), -phi.dx(1))), W.element.interpolation_points())
E.interpolate(expr)

In [16]:
# Use multiphenics to plot the vector field.
mpio.plot_vector_field(E,name="Electric Field", glyph_factor=0.5)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [17]:
# Store this solution for reuse later.
phi_initial = phi.copy()

# Slow Relaxation

We now let the system relax to equilibrium.  If the system is weakly conducting, then relaxation to equilibrium is slow enough that conduction and displacement currents cancel each other and there is no magnetic field.  (See Heald & Marion 4.4.)

In this case, the problem is "quasi-static".  We can solve the time *dependent* problem by solving a time *independent* problem at each time step, using the current value of $rho(t)$.

The continuity equation and Ohm's Law imply that
$$\rho(t) = \rho_0 \, e^{-t/\tau}$$
where the relaxation time $\tau$ is
$$\tau = \dfrac{4\pi\sigma}{\epsilon}$$.

We can visualize this by placing the FEM solving code inside of a loop.

In [18]:
# This is the time step for the simulation.
dt = fem.Constant(domain, 0.05)

# Initial time and maximum time.
t0 = 0.0
t_max = 10*tau

In [19]:
# Constant factor from Maxwell's equations.
k = fem.Constant(domain, 4*np.pi)

# This is the problem we are going to solve.
F = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
F -= k * ufl.inner(rho, v) * ufl.dx
(a, L) = ufl.system(F)

In [20]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

In [21]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

In [22]:
# Now we use the PETSc problem to construct the linear algebra problem.
solver = petsc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(petsc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(petsc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

In [23]:
import pyvista
# Start virtual framebuffer for plotting.
pyvista.start_xvfb()

# Prepare the mesh for PyVista ...
topology, cells, geometry = plot.create_vtk_mesh(V)

In [24]:
# Create a plotter for charge density, but don't display the plot.
rho_file = "%03d-charge.jpg"

# It will be making a movie in the background.
rho_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
rho_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
rho_grid.point_data["rho"] = rho.x.array
rho_grid.set_active_scalars("rho")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
rho_warped = rho_grid.warp_by_scalar("rho", factor=10)

# Add the warped grid to the plot.
rho_plotter.add_mesh(
    rho_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
)

Actor (0x7f9512eeb4b0)
  Center:                     (0.0, 0.0, 3.884403944015503)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -5.000E+00, 5.000E+00
  Y Bounds                    -5.000E+00, 5.000E+00
  Z Bounds                    0.000E+00, 7.769E+00
  User matrix:                Unset
  Has mapper:                 True

Property (0x7f9512eebd70)
  Ambient:                     0.0
  Ambient color:               Color(name='white', hex='#ffffffff')
  Anisotropy:                  0.0
  Color:                       Color(name='white', hex='#ffffffff')
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='white', hex='#ffffffff')
  Edge color:                  Color(name='black', hex='#000000ff')
  Interpolation:               "Flat"
  Lighting:                    False
  Line width

In [25]:
# Do exactly the same thing for the potential.
phi_file = "%03d-potential.jpg"
phi_plotter = pyvista.Plotter(off_screen=True, notebook=False)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array
phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
)

Actor (0x7f95128804b0)
  Center:                     (0.0, 0.0, 6.0826334953308105)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -5.000E+00, 5.000E+00
  Y Bounds                    -5.000E+00, 5.000E+00
  Z Bounds                    0.000E+00, 1.217E+01
  User matrix:                Unset
  Has mapper:                 True

Property (0x7f9512880670)
  Ambient:                     0.0
  Ambient color:               Color(name='white', hex='#ffffffff')
  Anisotropy:                  0.0
  Color:                       Color(name='white', hex='#ffffffff')
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='white', hex='#ffffffff')
  Edge color:                  Color(name='black', hex='#000000ff')
  Interpolation:               "Flat"
  Lighting:                    False
  Line widt

In [ ]:
n = 0
# Take final snapshots for the video.
rho_frame = rho_file % n
print(rho_frame)
rho_plotter.show(screenshot=rho_frame, auto_close=False)
# phi_frame = phi_file % n
# phi_plotter.screenshot(filename=rho_frame, return_img=False)

In [ ]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = t0
n = 0

# Set up the initial potential.
phi = phi_initial.copy()

# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    rho_frame = rho_file % n
    rho_plotter.show(screenshot=rho_frame, auto_close=False)
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt.value
    rho.interpolate(rho_function(t))
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, phi.vector)

    # Distribute the solution to all processes.
    phi.x.scatter_forward()

    # Update un --- the current value of the function.
    phi_n.x.array[:] = phi.x.array

    # Update the plots and save the frame to the GIF.
    rho_grid.point_data["rho"] = rho.x.array
    rho_warped = rho_grid.warp_by_scalar("rho", factor=10)
    rho_plotter.update_scalars(rho.x.array, render=False)
    rho_plotter.update_coordinates(rho_warped.points)

    phi_grid.point_data["phi"] = phi.x.array
    phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
    phi_plotter.update_scalars(phi.x.array, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)
    phi_frame = phi_file % n

    n += 1

# Take final snapshots for the video.
rho_frame = rho_file % n
rho_plotter.show(screenshot=rho_frame, auto_close=False)
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
rho_plotter.close()
phi_plotter.close()

In [ ]:
!ffmpeg -y -i %03d-charge.jpg -pix_fmt yuv420p charge-movie.mp4
!ffmpeg -y -i %03d-potential.jpg -pix_fmt yuv420p potential-movie.mp4

In [ ]:
# Get tools for embedding movies in CoLab.
from IPython.display import HTML
from base64 import b64encode

In [ ]:
# Play the charge movie.
mp4 = open('charge-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# Play the potential movie.
mp4 = open('charge-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)